In [1]:
!sudo apt update
!sudo apt install libcairo2-dev ffmpeg \
    texlive texlive-latex-extra texlive-fonts-extra \
    texlive-latex-recommended texlive-science \
    tipa libpango1.0-dev
!pip install manim
!pip install IPython --upgrade

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:6 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:13 http://security.ubuntu.c

     |████████████████████████████████| 793 kB 25.2 MB/s 
     |████████████████████████████████| 381 kB 49.9 MB/s 
  Attempting uninstall: prompt-toolkit
    Found existing installation: prompt-toolkit 1.0.18
    Uninstalling prompt-toolkit-1.0.18:
      Successfully uninstalled prompt-toolkit-1.0.18
  Attempting uninstall: IPython
    Found existing installation: ipython 5.5.0
    Uninstalling ipython-5.5.0:
      Successfully uninstalled ipython-5.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyter-console 5.2.0 requires prompt-toolkit<2.0.0,>=1.0.0, but you have prompt-toolkit 3.0.29 which is incompatible.
google-colab 1.0.0 requires ipython~=5.5.0, but you have ipython 7.33.0 which is incompatible.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.


In [ ]:
from manim import *

Manim Community v0.14.0

In [ ]:
%%manim -qk -v WARNING scene_1

config.media_width = "80%"
import sympy as sp
import numpy as np

class scene_1(Scene):
    def construct(self):
        def velocity_x(t, v_vek, ro, m, A, Cd):
            g = 9.80665
            v1,v2 = v_vek[0], v_vek[1]
            vx1 = v2
            vx2 = - (Cd*ro*A*(v2**2))/(2*m)
            return vx1, vx2

        def velocity_y(t, v_vek, ro, m, A, Cd):
            g = 9.80665
            v1,v2 = v_vek[0], v_vek[1]
            vy1 = v2
            if v2>0:
              vy2 = - g - (Cd*ro*A*(v2**2))/(2*m)
            else:
              vy2 = - g + (Cd*ro*A*(v2**2))/(2*m)
            return vy1, vy2

        def RK4(initial, lower, upper, h, ro, m, A, Cd, diff_func):
            t = np.arange(lower, upper, h)
            n, dim = len(t), len(initial)
            yvek = np.zeros([n, dim])
            k1v, k2v, k3v, k4v = np.zeros([n, dim]), np.zeros([n, dim]), np.zeros([n, dim]), np.zeros([n, dim])
            yvek[0, :] = initial
            for i in range(n-1):
              k1v[i, :] = h * np.array(diff_func(t[i], yvek[i, :], ro, m, A, Cd))
              k2v[i, :] = h * np.array(diff_func(t[i] + (1/2)*h, yvek[i, :] + (1/2)*k1v[i, :], ro, m, A, Cd))
              k3v[i, :] = h * np.array(diff_func(t[i] + (1/2)*h, yvek[i, :] + (1/2)*k2v[i, :], ro, m, A, Cd))
              k4v[i, :] = h * np.array(diff_func(t[i] + h, yvek[i, :] + k3v[i, :], ro, m, A, Cd))
              yvek[i+1, :] = yvek[i, :] + (1/6)*(k1v[i, :] + 2*k2v[i, :] + 2*k3v[i, :] + k4v[i, :])
            return yvek

        def position(x_initial, y_initial, lower, upper, h, ro, m, A, Cd, vx_func, vy_func):
            g = 9.80665
            x = RK4(x_initial, lower, upper, h, ro, m, A, Cd, vx_func)
            y = RK4(y_initial, lower, upper, h, ro, m, A, Cd, vy_func)
            pos = np.zeros([len(y), 3])
            velx = np.zeros([len(x), 3])
            vely = np.zeros([len(y), 3])
            i = 0
            JUM = 0
            while i < len(y):
              z = 0 
              if y[i,0] >= 0:
                pos[i] = [x[i,0], y[i,0], z]
              elif y[i-1,0]*y[i,0] < 0:
                pos[i] = [x[i,0], 0, z]
                JUM = i+1
              else:
                pos[i] = [x[i,0], 0, z]
              vely[i] = [x[i,0], y[i,1], z]
              i += 1
            i = 0
            while i < len(x):
              z = 0 
              velx[i] = [x[i,0], x[i,1], z]
              i += 1
            if JUM==0:
              Pos = pos
              Vely = vely
              Velx = velx
            else: 
              Pos = pos[:JUM]
              Vely = vely[:JUM]
              Velx = velx[:JUM]
            return Pos, Velx, Vely

        ax = NumberPlane(
            x_range=[0, 24, 2],
            y_range=[0, 24, 2],
            x_length=12,
            y_length=6,
            background_line_style={
                "stroke_color": BLUE,
                "stroke_width": 3,
                "stroke_opacity": 0.2
            },
            axis_config={
                "include_tip": False,
                "include_numbers": True
            },
            y_axis_config={
                "label_direction":UL
            }
        )
        ax2 = NumberPlane(
            x_range=[0, 24, 2],
            y_range=[-48, 48, 8],
            x_length=12,
            y_length=6,
            background_line_style={
                "stroke_color": BLUE,
                "stroke_width": 3,
                "stroke_opacity": 0.2
            },
            axis_config={
                "include_tip": False,
                "include_numbers": True
            },
            y_axis_config={
                "label_direction":UL
            }
        )
        lower = 0
        upper = 15
        h = 0.01
        m = 10
        A = PI*(1**2)
        Cd = np.random.uniform(0.5, 0.7)
        ro = 1.81
        g = 9.80665
        v0 = 50
        theta0 = PI/3
        y_init = [5, v0*np.sin(theta0)]
        x_init = [0, v0*np.cos(theta0)]
        POS, VEL_X, VEL_Y = position(x_init, y_init, lower, upper, h, ro, m, A, Cd, velocity_x, velocity_y)
        POS1, VEL_X1, VEL_Y1 = position([0,50], [10,0], lower, upper, h, ro, m, A, Cd, velocity_x, velocity_y)
        POS2, VEL_X2, VEL_Y2 = position([0,30], [0,30], lower, upper, h, ro, m, A, Cd, velocity_x, velocity_y)
        func = VGroup()
        func.add(ParametricFunction(lambda t : ax.coords_to_point(*POS[np.uint(t)]), 
                                    t_range=np.array([0, len(POS)-1, 1]), use_smoothing=False).set_color(GREEN))
        func.add(ParametricFunction(lambda t : ax2.coords_to_point(*VEL_X[np.uint(t)]), 
                                    t_range=np.array([0, len(VEL_X)-1, 1]), use_smoothing=False).set_color(YELLOW))
        func.add(ParametricFunction(lambda t : ax2.coords_to_point(*VEL_Y[np.uint(t)]), 
                                    t_range=np.array([0, len(VEL_Y)-1, 1]), use_smoothing=False).set_color(BLUE))
        func1 = VGroup()
        func1.add(ParametricFunction(lambda t : ax.coords_to_point(*POS1[np.uint(t)]), 
                                    t_range=np.array([0, len(POS1)-1, 1]), use_smoothing=False).set_color(GREEN_A))
        func1.add(ParametricFunction(lambda t : ax2.coords_to_point(*VEL_X1[np.uint(t)]), 
                                    t_range=np.array([0, len(VEL_X1)-1, 1]), use_smoothing=False).set_color(YELLOW_A))
        func1.add(ParametricFunction(lambda t : ax2.coords_to_point(*VEL_Y1[np.uint(t)]), 
                                    t_range=np.array([0, len(VEL_Y1)-1, 1]), use_smoothing=False).set_color(BLUE_A))
        func2 = VGroup()
        func2.add(ParametricFunction(lambda t : ax.coords_to_point(*POS2[np.uint(t)]), 
                                    t_range=np.array([0, len(POS2)-1, 1]), use_smoothing=False).set_color(GREEN_D))
        func2.add(ParametricFunction(lambda t : ax2.coords_to_point(*VEL_X2[np.uint(t)]), 
                                    t_range=np.array([0, len(VEL_X2)-1, 1]), use_smoothing=False).set_color(YELLOW_D))
        func2.add(ParametricFunction(lambda t : ax2.coords_to_point(*VEL_Y2[np.uint(t)]), 
                                    t_range=np.array([0, len(VEL_Y2)-1, 1]), use_smoothing=False).set_color(BLUE_D))
        self.add(ax)
        self.play(Create(func[0], rate_func=rate_functions.linear),
                  Create(func1[0], rate_func=rate_functions.linear),
                  Create(func2[0], rate_func=rate_functions.linear),
                  run_time=4)
        self.play(FadeOut(func[0]),FadeOut(func1[0]),FadeOut(func2[0]), FadeOut(ax))
        self.wait()
        self.play(Create(ax2))
        self.play(Create(func[1], rate_func=rate_functions.linear),
                  Create(func[2], rate_func=rate_functions.linear),
                  Create(func1[1], rate_func=rate_functions.linear),
                  Create(func1[2], rate_func=rate_functions.linear),
                  Create(func2[1], rate_func=rate_functions.linear),
                  Create(func2[2], rate_func=rate_functions.linear),
                  run_time=4)
        self.wait()

In [ ]:
import manimpango
manimpango.list_fonts()
print(len(manimpango.list_fonts()))

205


In [ ]:
%%manim -qm -v WARNING scene_2

config.media_width = "80%"
import sympy as sp
import numpy as np
from functools import reduce

class scene_2(Scene):
    def construct(self):
        def fibbonaci(n):
            seq = np.zeros([n])
            seq[0] = 1
            seq[1] = 1
            for i in range(2,n):
              seq[i] = seq[i-1] + seq[i-2]
            return seq

        def factors(n):    
            return set(reduce(list.__add__, ([i, n//i] for i in range(1, int(n**0.5) + 1) if n % i == 0)))

        def seq_factors(n):
            seq = np.zeros([n-1])
            for j in range(1,n):
              fact = factors(j)
              seq[j-1] = sum(fact)
            return seq

        def SieveOfEratosthenes(n):
            prime = [True for i in range(n+1)]
            p = 2
            while(p * p <= n):
              if (prime[p] == True):
                for i in range(p * p, n + 1, p):
                  prime[i] = False
              p += 1
            X = []
            for p in range(2, n):
              if prime[p]:
                X.append(p)
            return X

        #intro seq_1
        text1 = Tex("Plotting in polar coordinates")
        tex1 = MathTex(r"(p,p)\;\text{ Where p is prime number}")
        grup1 = VGroup(text1, tex1).arrange(DOWN)
        self.play(Write(grup1), run_time=3)
        self.wait()
        self.play(Unwrite(grup1))
        self.wait()
        #seq_1
        seq_1 = SieveOfEratosthenes(85000)
        ax1 = Axes(
            x_range=[-75000, 75000, 10000],
            y_range=[-40000, 40000, 10000],
            x_length=15,
            y_length=8,
            axis_config={"include_tip": False}
        )
        P1 = VGroup()
        for prime in seq_1:
          P1.add(Dot(radius=0.015).set_color(BLUE).move_to(
              ax1.coords_to_point(
                  prime*np.cos(prime),
                  prime*np.sin(prime),
                  0
                  )
              )
          )
        self.play(Create(ax1))
        self.wait()
        self.play(Create(P1), run_time=3)
        self.wait()
        self.play(FadeOut(ax1), FadeOut(P1), run_time=2)
        #intro seq_2
        text2 = Tex("Plotting in polar coordinates")
        tex2 = MathTex(r"(\sigma(n),\sigma(n))\;\text{ Where }\sigma(n)\text{ is sum every divisors of n}")
        grup2 = VGroup(text2, tex2).arrange(DOWN)
        self.play(Write(grup2), run_time=3)
        self.wait()
        self.play(Unwrite(grup2))
        self.wait()
        #seq_2
        seq_2 = seq_factors(50000)
        ax2 = Axes(
            x_range=[-75000, 75000, 10000],
            y_range=[-40000, 40000, 10000],
            x_length=15,
            y_length=8,
            axis_config={"include_tip": False}
        )
        print(seq_2)
        P2 = VGroup()
        for num in seq_2:
          P2.add(Dot(radius=0.015).set_color(BLUE).move_to(
              ax2.coords_to_point(
                  num*np.cos(num),
                  num*np.sin(num),
                  0
                  )
              )
          )
        self.play(Create(ax2))
        self.wait()
        self.play(Create(P2), run_time=3)
        self.wait()
        self.play(FadeOut(ax2), FadeOut(P2), run_time=2)


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
%%manim -qm -v WARNING scene_3

config.media_width = "80%"
import sympy as sp
import numpy as np
from functools import reduce

class Bacteria(Dot):
    def __init__(self, point=ORIGIN, ** kwargs):
        Dot.__init__(self, point=point, color=GREEN, ** kwargs)
        self.velocity = 6 * np.random.random_sample(3) - 3 # [-3, 3] interval
        
class scene_3(Scene):
    def construct(self):
        # Create objects
        box = Rectangle(width=5, height=5)
        self.play(FadeIn(box))

        bacteria_array = []
        for i in range(0, 3):
            bacteria_position = 4 * np.random.random_sample((3,)) - 2 # [-2, 2] interval
            bacteria_array.append(Bacteria(bacteria_position))
            self.play(FadeIn(bacteria_array[i]))

        # Collision detection
        def update_bacteria(bacteria, dt):
            bacteria.acceleration = 2 * np.random.random_sample(3) - 1 # [-1, 1] interval
            bacteria.velocity = bacteria.velocity + bacteria.acceleration * dt
            bacteria.shift(bacteria.velocity * dt)

            # Bounce off walls
            if bacteria.get_left()[0] <= box.get_left()[0] or bacteria.get_right()[0] >= box.get_right()[0]:
                bacteria.velocity[0] = -bacteria.velocity[0]

            # Bounce off ground and roof
            if bacteria.get_bottom()[1] <= box.get_bottom()[1] or bacteria.get_top()[1] >= box.get_top()[1]:
                bacteria.velocity[1] = -bacteria.velocity[1]

        # Attach collision detection to updater
        for bacteria in bacteria_array:
            bacteria.add_updater(update_bacteria)
        self.wait(20)

In [ ]:
%%manim -qm -v WARNING scene_00

config.media_width = "80%"
import sympy as sp
import numpy as np

class scene_00(Scene):
    def construct(self):
        text1 = Text("Bird fly up to the sky")
        text2 = Text("")